<a href="https://colab.research.google.com/github/jgdshkovi/Benn/blob/master/vgg_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/Z/filter_pruning

/content/drive/My Drive/Z/filter_pruning


In [0]:
import argparse
import numpy as np
import os

import math
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

from models import *
import spherical_kmeans as skm

In [0]:
test_batch_size = 256
dataset = 'cifar10'
cfg = [32, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 256, 256, 256, 'M', 256, 256, 256]
filtered_cfg = [16, 32, 'M', 64, 64, 'M', 128, 128, 128, 'M', 128, 128, 128, 'M', 128, 128, 128]
cuda = True

In [0]:
model = torch.load('L1_logs/model_best.pth.tar')
checkpoint = torch.load('L1_logs/checkpoint.pth.tar')

In [35]:
model = vgg(dataset='cifar10', depth=16,cfg=cfg)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:
def test(model):
    kwargs = {'num_workers': 1, 'pin_memory': True} if True else {}
    if dataset == 'cifar10':
        test_loader = torch.utils.data.DataLoader(
            datasets.CIFAR10('./data.cifar10', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    elif dataset == 'cifar100':
        test_loader = torch.utils.data.DataLoader(
            datasets.CIFAR100('./data.cifar100', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    else:
        raise ValueError("No valid dataset is given.")
    model.eval()
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.1f}%)\n'.format(
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))

In [0]:
from collections import defaultdict
def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() if len(locs)>1)


def return_mask(labels):
	modsource = labels.copy()
	for dup in sorted(list_duplicates(labels)):
	    lis = dup[1][1:]
	    for i in lis:
	    	modsource[i] = -1

	mask = []
	for el in modsource:
		if el!=-1:
			mask.append(1)
		else:
			mask.append(0)
	return mask


def return_cluster_labels(feat_wts,no_of_clus):
  res = skm.spherical_k_means(feat_wts,n_clusters=no_of_clus,random_state=10)
  return res[1]

In [37]:
cfg_mask = []
layer_id = 0
for m in model.modules():
  if isinstance(m , nn.Conv2d):
    shape = m.weight.data.shape
    print(shape)
    reshaped_tensor = m.weight.data.clone().numpy().reshape(shape[0] , shape[1]*shape[2]*shape[3])

    labels = return_cluster_labels(reshaped_tensor,filtered_cfg[layer_id])
    mask = return_mask(labels)
    cfg_mask.append(torch.tensor(mask))
    layer_id += 1
  elif isinstance(m, nn.MaxPool2d):
    layer_id += 1

torch.Size([32, 3, 3, 3])
torch.Size([64, 32, 3, 3])
torch.Size([128, 64, 3, 3])
torch.Size([128, 128, 3, 3])
torch.Size([256, 128, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])


In [38]:
newmodel = vgg(dataset = 'cifar10' ,cfg=filtered_cfg)
newmodel.cuda()

start_mask = torch.ones(3)
layer_id_in_cfg = 0
end_mask = cfg_mask[layer_id_in_cfg]
for [m0, m1] in zip(model.modules(), newmodel.modules()):
    if isinstance(m0, nn.BatchNorm2d):
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx1.size == 1:
            idx1 = np.resize(idx1,(1,))
        m1.weight.data = m0.weight.data[idx1.tolist()].clone()
        m1.bias.data = m0.bias.data[idx1.tolist()].clone()
        m1.running_mean = m0.running_mean[idx1.tolist()].clone()
        m1.running_var = m0.running_var[idx1.tolist()].clone()
        layer_id_in_cfg += 1
        start_mask = end_mask
        if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
            end_mask = cfg_mask[layer_id_in_cfg]
    elif isinstance(m0, nn.Conv2d):
        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        print('In shape: {:d}, Out shape {:d}.'.format(idx0.size, idx1.size))
        if idx0.size == 1:
            idx0 = np.resize(idx0, (1,))
        if idx1.size == 1:
            idx1 = np.resize(idx1, (1,))
        w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
        w1 = w1[idx1.tolist(), :, :, :].clone()
        m1.weight.data = w1.clone()
    elif isinstance(m0, nn.Linear):
        if layer_id_in_cfg == len(cfg_mask):
            idx0 = np.squeeze(np.argwhere(np.asarray(cfg_mask[-1].cpu().numpy())))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            m1.weight.data = m0.weight.data[:, idx0].clone()
            m1.bias.data = m0.bias.data.clone()
            layer_id_in_cfg += 1
            continue
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()
    elif isinstance(m0, nn.BatchNorm1d):
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()
        m1.running_mean = m0.running_mean.clone()
        m1.running_var = m0.running_var.clone()

In shape: 3, Out shape 16.
In shape: 16, Out shape 32.
In shape: 32, Out shape 64.
In shape: 64, Out shape 64.
In shape: 64, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 128.


In [39]:
torch.save({'cfg': filtered_cfg, 'state_dict': newmodel.state_dict()},'Filter_logs/pruned.pth.tar')
print(newmodel)
model = newmodel
model.cuda()
acc = test(model)

num_parameters = sum([param.nelement() for param in newmodel.parameters()])
with open( "Filter_logs/prune.txt", "w") as fp:
    fp.write("Number of parameters: \n"+str(num_parameters)+"\n")
    fp.write("Test accuracy: \n"+str(acc)+"\n")

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [15]:
ls

cmds.xt.txt       L1_logs/          models/              vggprune.py
compute_flops.py  main_finetune.py  __pycache__/
data.cifar10/     Main_logs/        spherical_kmeans.py
Filter_logs/      main.py           Test_folder/


In [17]:
!python vggprune.py --dataset cifar10 --model Main_logs/model_best.pth.tar --save L1_logs/

=> loading checkpoint 'Main_logs/model_best.pth.tar'
=> loaded checkpoint 'Main_logs/model_best.pth.tar' (epoch 151) Prec1: 0.936300
Pre-processing Successful!
vggprune.py:74: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)

Test set: Accuracy: 9363/10000 (93.6%)

In shape: 3, Out shape 32.
In shape: 32, Out shape 64.
In shape: 64, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
vgg(
  (feature): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3)

In [42]:
!python main_finetune.py --refine Filter_logs/pruned.pth.tar --dataset cifar10 --epochs 15 --save Filter_logs/

Files already downloaded and verified
Train Epoch: 0 [0/50000 (0.0%)]	Loss: 3.237350
Train Epoch: 0 [6400/50000 (12.8%)]	Loss: 0.747668
Train Epoch: 0 [12800/50000 (25.6%)]	Loss: 0.534155
Train Epoch: 0 [19200/50000 (38.4%)]	Loss: 0.653303
Train Epoch: 0 [25600/50000 (51.2%)]	Loss: 0.263772
Train Epoch: 0 [32000/50000 (63.9%)]	Loss: 0.398468
Train Epoch: 0 [38400/50000 (76.7%)]	Loss: 0.427622
Train Epoch: 0 [44800/50000 (89.5%)]	Loss: 0.532588
main_finetune.py:153: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))

Test set: Average loss: 0.4762, Accuracy: 8476/10000 (84.8%)

Train Epoch: 1 [0/50000 (0.0%)]	Loss: 0.355136
Train Epoch: 1 [6400/50000 (12.8%)]	Loss: 0.268191
Train Epo

In [51]:
import compute_flops as flps

model = torch.load('L1_logs/model_best.pth.tar')
checkpoint = torch.load('L1_logs/checkpoint.pth.tar')
model = vgg(dataset='cifar10', depth=16,cfg=cfg)
model.load_state_dict(checkpoint['state_dict'])

flps.print_model_param_nums(model=model)
#flps.print_model_param_flops(model=model)

  + Number of params: 5.40M
